<a href="https://colab.research.google.com/github/marcospiau/ia368-dd-dl4ir/blob/main/aula05-doc2query/aula05_doc2query_indexing_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook will be used to index texts and eval results

# Installs and imports

In [1]:
!free -mh
!lscpu

              total        used        free      shared  buff/cache   available
Mem:           25Gi       588Mi        20Gi       1.0Mi       4.0Gi        24Gi
Swap:            0B          0B          0B
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cach

In [2]:
%%capture
!wget -nc https://raw.githubusercontent.com/marcospiau/ia368-dd-dl4ir/main/scripts/install_anserini.sh && chmod +x install_anserini.sh && time ./install_anserini.sh

In [3]:
%%capture
!pip install -q ftfy polars toolz cytoolz transformers datasets gcsfs
!pip install -U t5[gcp,cache-tasks]==0.9.3
!pip install -U jaxlib
!sudo apt install -qq tree htop

**PS.: notebook must be restarted after packages above are installed.**

# gcloud authentication

In [1]:
import os
from google.colab import auth

os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user(clear_output=False)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=25Ho79j8TO1yUvRyMNwZ6jwDuXXFDS&prompt=consent&access_type=offline&code_challenge=OkDPMswv0u0xQcUUUu1iamFg8M0tT5D1CSd7ZdYh1ho&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk47Rem08dpCMnt7JiA2KaUtdmWHiC_Nf1G6WaxDbstYnI2FwoeiVal3K1Unr2f-LA


In [4]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=DWk30D7DWurElZ7SNQZSY5dlCoGmOL&prompt=consent&access_type=offline&code_challenge=8lwHuKgDiX-yMEgLhIpn3Zh1kS_Gqf59W9-ErxIOilI&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk6fMAHCGVyc1AqrBLa-LQtOAMjEeDdhuxmXj84m3xpNtdGjjThthYQMVBG5RBeOsw

You are now logged in as [m136825@dac.unicamp.br].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


# Preparing qrels and queries files

## RASCUNHO, TIRAR!!

In [261]:
import datasets
import toolz
import multiprocessing as mp
import pandas as pd
from collections import Counter, defaultdict
import itertools
import polars as pl

import more_itertools
from etils import epath
import functools
from tqdm import tqdm

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

os.environ['POLARS_FMT_STR_LEN']='1000'

In [ ]:
from transformers import T5Tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def concat_title_and_text(ex):
    return {'title_and_text': f"{ex['title']}. {ex['text']}"}

def encode(batch):
    lengths = tokenizer(batch['title_and_text'], return_length=True)['length']
    return {'lengths': lengths}

In [ ]:
ds_corpus = datasets.load_dataset('BeIR/trec-covid', 'corpus')['corpus']
ds_corpus = ds_corpus.map(concat_title_and_text)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
token_lengths = pd.Series(ds_corpus.map(encode, batched=True, num_proc=mp.cpu_count())['lengths'])

In [ ]:
tokenizer(ds_corpus.select(range(1))['title_and_text'])

{'input_ids': [[14067, 753, 13, 1543, 18, 1409, 1926, 499, 509, 21178, 9, 30195, 15, 13315, 44, 2671, 28508, 9, 702, 172, 636, 4457, 6, 1022, 26, 26, 9, 107, 6, 11279, 13849, 5, 3, 10539, 683, 14196, 8087, 10, 100, 29825, 5059, 1132, 8788, 8, 29969, 1863, 11, 3739, 753, 13, 1283, 1221, 28, 1543, 18, 1409, 1926, 499, 509, 21178, 9, 30195, 15, 13315, 44, 2671, 28508, 9, 702, 172, 636, 4457, 6, 1022, 26, 26, 9, 107, 6, 11279, 13849, 5, 3, 24506, 6299, 3592, 10, 18027, 28, 1465, 283, 5, 30195, 15, 9757, 45, 19944, 19622, 7, 45, 1762, 6622, 190, 1882, 6260, 130, 4313, 190, 8, 5893, 6420, 6427, 3187, 5, 15054, 7, 13, 1221, 130, 9112, 5, 3, 12200, 4254, 4578, 10, 1283, 1221, 130, 4313, 6, 5400, 13642, 15967, 6210, 13, 4068, 831, 7209, 5, 1377, 13315, 14156, 15967, 6210, 130, 573, 18, 9, 75, 1169, 1271, 5, 37, 7952, 4161, 66, 1246, 1637, 68, 47, 167, 1017, 16, 9806, 7, 6918, 15967, 6210, 11, 554, 18, 6646, 502, 4743, 15967, 6210, 5, 94, 6935, 215, 18, 7775, 68, 47, 167, 1017, 16, 8, 1590, 41, 

In [ ]:
token_lengths.describe().to_frame().round(2)

,0
count,171332.00
mean,266.93
std,254.57
min,3.00
25%,57.00
50%,268.00
75%,406.00
max,42655.00


In [ ]:
token_lengths.gt(512).value_counts().to_frame()

,0
False,154953
True,16379


## Convert qrels to TREC format

In [7]:
ds_qrels = datasets.load_dataset('BeIR/trec-covid-qrels')['test']
ds_qrels

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___csv/BeIR--trec-covid-qrels-1766e3af5b0b856a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 66336
})

In [8]:
a = ds_qrels.to_pandas().apply(lambda x: (str(x['query-id']), str(x['corpus-id'])), axis=1)
a = sorted(a)
Counter(a).most_common(10)

[(('1', '005b2j4b'), 1),
 (('1', '00fmeepz'), 1),
 (('1', '0194oljo'), 1),
 (('1', '021q9884'), 1),
 (('1', '02f0opkr'), 1),
 (('1', '02say5f1'), 1),
 (('1', '0376d6vf'), 1),
 (('1', '047xpt2c'), 1),
 (('1', '04ftw7k9'), 1),
 (('1', '05vx82oo'), 1)]

In [9]:
b = pd.read_csv('tools/topics-and-qrels/qrels.beir-v1.0.0-trec-covid.test.txt', sep=' ', header=None).apply(lambda x: (str(x[0]), str(x[2])), axis=1)
b = sorted(b)


In [11]:
# There are two ids missing from qrels in anserini tools
print(set(a) - set(b))
print(set(b) - set(a))

{('50', 'svo94kuo'), ('38', '9hbib8b3')}
set()


In [19]:
def convert_qrels_to_trec_format(output_file):
    def to_line(ex):
        # qid 0 docid score
        return '{} 0 {} {}\n'.format(ex['query-id'], ex['corpus-id'],
                                   ex['score'])
    ds = datasets.load_dataset('BeIR/trec-covid-qrels')['test']
    with open(output_file, 'w') as f:
        for item in ds.to_list():
            line = to_line(item)
            f.write(line)

QRELS_FILE = 'trec-covid-qrels_trec_format.txt'

convert_qrels_to_trec_format(QRELS_FILE)
!wc -l {QRELS_FILE}
!head {QRELS_FILE}

  0%|          | 0/1 [00:00<?, ?it/s]

66336 trec-covid-qrels_trec_format.txt
1 0 005b2j4b 2
1 0 00fmeepz 1
1 0 g7dhmyyo 2
1 0 0194oljo 1
1 0 021q9884 1
1 0 02f0opkr 1
1 0 047xpt2c 0
1 0 04ftw7k9 0
1 0 pl9ht0d0 0
1 0 05vx82oo 0


## Convert queries to TREC format

~Queries are already in `anserini/src/main/resources/topics-and-qrels/topics.beir-v1.0.0-trec-covid.test.tsv.gz`, so we are going to use it instead of downloading from HuggingFace.~



In [20]:
def convert_queries_to_trec_format(output_file):
    ds = datasets.load_dataset('BeIR/trec-covid', 'queries')['queries']
    ds.to_csv(output_file, header=None, sep='\t', columns=['_id', 'text'])

QUERIES_FILE = 'trec-covid-queries_trec_format.txt'

convert_queries_to_trec_format(QUERIES_FILE)
!wc -l {QUERIES_FILE}
!cat {QUERIES_FILE}

  0%|          | 0/1 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

50 trec-covid-queries_trec_format.txt
1	what is the origin of COVID-19
2	how does the coronavirus respond to changes in the weather
3	will SARS-CoV2 infected people develop immunity? Is cross protection possible?
4	what causes death from Covid-19?
5	what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?
6	what types of rapid testing for Covid-19 have been developed?
7	are there serological tests that detect antibodies to coronavirus?
8	how has lack of testing availability led to underreporting of true incidence of Covid-19?
9	how has COVID-19 affected Canada
10	has social distancing had an impact on slowing the spread of COVID-19?
11	what are the guidelines for triaging patients infected with coronavirus?
12	what are best practices in hospitals and at home in maintaining quarantine?
13	what are the transmission routes of coronavirus?
14	what evidence is there related to COVID-19 super spreaders
15	how long can the coronavirus live outside the body
16	how long doe

# Download pre-built index from pyserini

Following instructions, we will download the pre-built index `beir-v1.0.0-trec-covid-flat`.

In [1]:
from pyserini.index.lucene import IndexReader
from pyserini.search.lucene import LuceneSearcher
from pyserini.util import download_prebuilt_index

Just download file:

In [3]:
PREBUILT_INDEX_DIR = download_prebuilt_index('beir-v1.0.0-trec-covid.flat',
                                             verbose=True)
print(f'PREBUILT_INDEX_DIR is {PREBUILT_INDEX_DIR}')
!ls -lht {PREBUILT_INDEX_DIR}

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:15, 14.3MB/s]                           


Extracting /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz into /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade...
PREBUILT_INDEX_DIR is /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade
total 258M
-rw-rw-r-- 1 2537 2603  340 Nov 16 15:21 _0.fnm
-rw-rw-r-- 1 2537 2603  19M Nov 16 15:21 _0_Lucene90_0.doc
-rw-rw-r-- 1 2537 2603  22M Nov 16 15:21 _0_Lucene90_0.pos
-rw-rw-r-- 1 2537 2603 3.6M Nov 16 15:21 _0_Lucene90_0.tim
-rw-rw-r-- 1 2537 2603  81K Nov 16 15:21 _0_Lucene90_0.tip
-rw-rw-r-- 1 2537 2603  306 Nov 16 15:21 _0_Lucene90_0.tmd
-rw-rw-r-- 1 2537 2603  516 Nov 16 15:21 _0.si
-rw-rw-r-- 1 2537 2603  154 Nov 16 15:21 segments_1
-rw-rw-r-- 1 2537 2603  242 Nov 16 15:21 _0.fdm
-rw-rw-r-- 1 2537 2603 127M Nov 16 15:21 _0.fdt
-rw-rw-r-- 1 2537 2603  12K Nov 16 15:21 _0.fdx
-rw-rw-r-- 1 2537 2603 1.4M 

Load index and show statistics:

In [4]:
index_reader = IndexReader.from_prebuilt_index('beir-v1.0.0-trec-covid.flat',
                                               verbose=True)
index_reader

Attempting to initialize pre-built index beir-v1.0.0-trec-covid.flat.
/root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade already exists, skipping download.
Initializing beir-v1.0.0-trec-covid.flat...
{'total_terms': 20822821, 'documents': 171331, 'non_empty_documents': 171331, 'unique_terms': 202648}
Index passes consistency checks against pre-built index 'beir-v1.0.0-trec-covid.flat'!


# Validating results from anserini regressions on Beir TREC Covid

We will first attempt to replicate Anserini's results for TREC-COVID. Essentially, we will follow the instructions provided in [Anserini Regressions: BEIR (v1.0.0) — TREC-COVID](https://github.com/castorini/anserini/blob/master/docs/regressions-beir-v1.0.0-trec-covid-flat.md).

Another important file is this YAML, which contains information about evaluation and index building: https://github.com/castorini/anserini/blob/master/src/main/resources/regression/beir-v1.0.0-trec-covid-flat.yaml

In [26]:
import shlex
import subprocess

TREC_EVAL_BIN_PATH = './tools/eval/trec_eval.9.0.4/trec_eval'
# Essa função foi escrita usando o github copilot
def get_trec_eval_metrics(flags, qrels_path, results_path):
    """Runs trec_eval and returns the results as a dictionary.

    Args:
        flags (str): Flags to pass to trec_eval.
        qrels_path (str): Path to the qrels file.
        results_path (str): Path to the results file.

    Returns:
        Dict[str, float]: A dictionary mapping metric names to their values.
    """
    output = subprocess.check_output([
        TREC_EVAL_BIN_PATH,
        qrels_path,
        results_path,
        *shlex.split(flags)
    ]).decode('utf-8')
    return {
        line.split()[0]: (line.split()[2])
        for line in output.splitlines()
    }
# https://github.com/castorini/anserini/blob/master/docs/regressions-beir-v1.0.0-trec-covid-flat.md
# https://github.com/castorini/anserini/blob/master/src/main/resources/regression/beir-v1.0.0-trec-covid-flat.yaml



In [13]:
!head  trec-covid-queries_trec_format.txt

1	what is the origin of COVID-19
2	how does the coronavirus respond to changes in the weather
3	will SARS-CoV2 infected people develop immunity? Is cross protection possible?
4	what causes death from Covid-19?
5	what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?
6	what types of rapid testing for Covid-19 have been developed?
7	are there serological tests that detect antibodies to coronavirus?
8	how has lack of testing availability led to underreporting of true incidence of Covid-19?
9	how has COVID-19 affected Canada
10	has social distancing had an impact on slowing the spread of COVID-19?


In [23]:
# -index /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade \
# -topics anserini/src/main/resources/topics-and-qrels/topics.beir-v1.0.0-trec-covid.test.tsv.gz \
!mkdir pv runs
!anserini/target/appassembler/bin/SearchCollection \
-index {PREBUILT_INDEX_DIR} \
-topics trec-covid-queries_trec_format.txt \
-topicreader TsvString \
-output runs/run.beir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test.txt \
-bm25 -removeQuery -hits 1000

2023-04-10 03:35:09,894 INFO  [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============
2023-04-10 03:35:09,899 INFO  [main] search.SearchCollection (SearchCollection.java:951) - Index: /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade
2023-04-10 03:35:10,210 INFO  [main] search.SearchCollection (SearchCollection.java:955) - Fields: []
2023-04-10 03:35:10,212 INFO  [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer
2023-04-10 03:35:10,213 INFO  [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter
2023-04-10 03:35:10,214 INFO  [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false
2023-04-10 03:35:10,215 INFO  [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null
2023-04-10 03:35:10,252 INFO  [main] search.SearchCollection (SearchCollection.java:1227)

In [27]:
#  -c: Average over the complete set of queries in the relevance judgements  
#      instead of the queries in the intersection of relevance judgements 
#      and results.  Missing queries will contribute a value of 0 to all 
#      evaluation measures (which may or may not be reasonable for a  
#      particular evaluation measure, but is reasonable for standard TREC
#      measures.) Default is off.
get_trec_eval_metrics('-c -m ndcg_cut.10', 'trec-covid-qrels_trec_format.txt', 'runs/run.beir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test.txt')

{'ndcg_cut_10': '0.5947'}

The value for ndcg_cut_10 obtained match the expected one.

# Results using expansion

Results for expanded queries

## Downloading files locally

In [29]:
!mkdir -pv ids_and_contents generated_queries
!gsutil -m rsync -r gs://marcospiau-doc2query-aula05/ids_and_contents ids_and_contents
!gsutil -m rsync -r gs://marcospiau-doc2query-aula05/generated_queries generated_queries

Building synchronization state...
Starting synchronization...
Copying gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/id.txt...
Copying gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt...
/ [2/2 files][185.1 MiB/185.1 MiB] 100% Done                                    
Operation completed over 2 objects/185.1 MiB.                                    
Building synchronization state...
Starting synchronization...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/one_line_per_document_beam_search.log...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/one_line_per_document_topk_sampling.log...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/predictions.txt-1005516...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-15.txt-1005516...
Copyin

In [30]:
!tree -lht ids_and_contents
!tree -lht generated_queries

ids_and_contents
└── [4.0K]  one_line_per_document
    ├── [184M]  content.txt
    └── [1.5M]  id.txt

1 directory, 2 files
generated_queries
├── [4.0K]  one_line_per_document_beam_search
│   ├── [6.8M]  predictions.txt-1005516
│   └── [412K]  one_line_per_document_beam_search.log
├── [4.0K]  one_line_per_document_greedy_decode
│   ├── [6.9M]  predictions.txt-1005516
│   └── [412K]  one_line_per_document_greedy_decode.log
└── [4.0K]  one_line_per_document_topk_sampling
    ├── [6.9M]  predictions-iter-01.txt-1005516
    ├── [7.0M]  predictions-iter-02.txt-1005516
    ├── [6.9M]  predictions-iter-03.txt-1005516
    ├── [7.0M]  predictions-iter-04.txt-1005516
    ├── [7.0M]  predictions-iter-05.txt-1005516
    ├── [7.0M]  predictions-iter-06.txt-1005516
    ├── [7.0M]  predictions-iter-07.txt-1005516
    ├── [7.0M]  predictions-iter-08.txt-1005516
    ├── [7.0M]  predictions-iter-09.txt-1005516
    ├── [7.0M]  predictions-iter-10.txt-1005516
    ├── [7.0M]  predictions-iter-11.txt-100551

There are unk tokens on generated queries:

In [135]:
!grep -ir ⁇ generated_queries | head

generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:what does the hairpin structure of dna polymerase  ⁇  mRNA bind to?
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:what is IFN- ⁇  response to adenovirus
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516: ⁇  secretase activity inhibited gene transcription
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:does RRV induce IFN ⁇  production
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:how long does Ifn- ⁇  produce in spleen
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:does lack of  ⁇ -casein cause weight loss
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:IFN- ⁇  synthesis by macrophage
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:I ⁇ B

Replacing with sed:

In [138]:
!grep -ir ⁇ generated_queries | head | sed s,'⁇',' ',g

generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:what does the hairpin structure of dna polymerase     mRNA bind to?
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:what is IFN-    response to adenovirus
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:    secretase activity inhibited gene transcription
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:does RRV induce IFN    production
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:how long does Ifn-    produce in spleen
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:does lack of    -casein cause weight loss
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:IFN-    synthesis by macrophage
generated_queries/one_line_per_document_topk_sampling/predictions-iter-17.txt-1005516:I   B

In [157]:
!ls -lht 

total 2.6M
drwxr-xr-x  5 root root  4.0K Apr 10 03:52 generated_queries
drwxr-xr-x  3 root root  4.0K Apr 10 03:52 ids_and_contents
-rw-r--r--  1 root root  1.5M Apr 10 03:44 id.txt
drwxr-xr-x  2 root root  4.0K Apr 10 03:35 runs
drwxr-xr-x  2 root root  4.0K Apr 10 03:35 pv
-rw-r--r--  1 root root  3.6K Apr 10 03:34 trec-covid-queries_trec_format.txt
-rw-r--r--  1 root root 1023K Apr 10 03:34 trec-covid-qrels_trec_format.txt
-rw-r--r--  1 root root  3.6K Apr 10 03:19 aa
drwxr-xr-x 13 root root  4.0K Apr 10 03:05 anserini
drwxr-xr-x 15 root root  4.0K Apr 10 03:04 pyserini
drwxr-xr-x  6 root root  4.0K Apr 10 03:03 tools
-rwxr-xr-x  1 root root  1011 Apr 10 03:03 install_anserini.sh
drwxr-xr-x  1 root root  4.0K Apr  6 13:39 sample_data


## Common functions

In [169]:
def load_ids(path):
    lines = []
    for line in open(path, 'r'):
        lines.append(line.strip())
    return lines

load_ids('ids_and_contents/one_line_per_document/id.txt')[:10]

['ug7v899j',
 '02tnwd4m',
 'ejv2xln0',
 '2b73a28n',
 '9785vg6d',
 'zjufx4fo',
 '5yhe786e',
 '8zchiykl',
 '8qnrcgnk',
 'jg13scgo']

In [224]:
def read_pred_files(*paths):
    def read_one(path):
        lines = []
        for line in open(path, 'r'):
            line = line.strip()
            line = line.replace('⁇', ' ')
            lines.append(line)
        return lines

    preds = list(map(read_one, paths))

    # assert all files have the same number of lines
    assert len(set(map(len, preds))) == 1
    # merge lines
    preds = list(zip(*preds))
    return preds

read_pred_files(*epath.Path('generated_queries/one_line_per_document_topk_sampling').glob('predictions-*'))[:10]

[('how many patients died of pneumonia',
  'how many patients died from pneumonia',
  'how many children die from pneumonia',
  'how many patients died from m pneumonia',
  'characteristics of mycoplasma pneumoniae in children',
  'series of prospective clinical trials on m pneumoniae',
  'how many patients died from pneumonia',
  'how many patients died from pneumonia',
  'how many patients died from pneumonia',
  'case series of mycoplasma pneumoniae',
  'how many patients died from m pneumoniae',
  'when did the pneumonia come',
  'how many patients died from m pneumoniae',
  'characteristics of M pneumoniae',
  'how many children died from pneumonia',
  'what is the incidence of pneumonia in children',
  'frequency of pneumonia in pre-school children',
  'characteristics of mycoplasma pneumoniae',
  'characteristics of mycoplasma pneumoniae',
  'how many patients died from pneumonia'),
 ('which of the following is an inflammatory mediator associated with lung injury?',
  'does nitr

In [225]:
def load_ids_and_preds(ids_path, *preds_paths):
    ids = load_ids(ids_path)
    preds = read_pred_files(*preds_paths)
    assert len(ids) == len(preds)
    # ids can be duplicated, so we will groupby id and concat all preds for it
    # ids are sorted by construction
    out = {}
    for key, group in itertools.groupby(iterable=zip(ids, preds),
                                        key=lambda x: x[0]):
        out[key] = list(more_itertools.flatten(toolz.pluck(1, group)))
    return out

list(itertools.islice(load_ids_and_preds('ids_and_contents/one_line_per_document/id.txt', *epath.Path('generated_queries/one_line_per_document_topk_sampling').glob('predictions-*')).items(), 10))

[('ug7v899j',
  ['how many patients died of pneumonia',
   'how many patients died from pneumonia',
   'how many children die from pneumonia',
   'how many patients died from m pneumonia',
   'characteristics of mycoplasma pneumoniae in children',
   'series of prospective clinical trials on m pneumoniae',
   'how many patients died from pneumonia',
   'how many patients died from pneumonia',
   'how many patients died from pneumonia',
   'case series of mycoplasma pneumoniae',
   'how many patients died from m pneumoniae',
   'when did the pneumonia come',
   'how many patients died from m pneumoniae',
   'characteristics of M pneumoniae',
   'how many children died from pneumonia',
   'what is the incidence of pneumonia in children',
   'frequency of pneumonia in pre-school children',
   'characteristics of mycoplasma pneumoniae',
   'characteristics of mycoplasma pneumoniae',
   'how many patients died from pneumonia']),
 ('02tnwd4m',
  ['which of the following is an inflammatory me

## Loading data

Data is small enough to be loaded in memory at once, so we will keep it simple and load all data in a single dataframe:

In [274]:
# single dataframe 
df_corpus_preds = pl.from_arrow(datasets.load_dataset(
    'BeIR/trec-covid','corpus')['corpus'].data.table)

# concat title and text - this is the baseline text for retrieval
# PS.: during query generation, we used the format
# pl.format('{}. {}', 'title', 'text').alias('title_and_text') to feed T5
# Here, we will simple concatenate the fields using a space

df_corpus_preds = df_corpus_preds.with_columns(
    pl.concat_str('title', 'text', separator=' ').alias('title_and_text'))
df_corpus_preds.head()

  0%|          | 0/1 [00:00<?, ?it/s]

_id,title,text,title_and_text
str,str,str,str
"""ug7v899j""","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia""","""OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were …","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (3…"
"""02tnwd4m""","""Nitric oxide: a pro-inflammatory mediator in lung disease?""","""Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the lun…","""Nitric oxide: a pro-inflammatory mediator in lung disease? Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be

Adding generated queries:

In [271]:
preds_paths_map = {
    x.name: ('ids_and_contents/one_line_per_document/id.txt',
             sorted(x.glob('predictions*')))
    for x in epath.Path('generated_queries').iterdir()
}
preds_paths_map

{'one_line_per_document_topk_sampling': ('ids_and_contents/one_line_per_document/id.txt',
  [PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-01.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-02.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-03.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-04.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-05.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-06.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-07.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-08.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictio

In [275]:
df_corpus_preds = df_corpus_preds.with_columns(
    pl.col('_id').map_dict(load_ids_and_preds(v[0], *v[1])).alias(f'preds_{k}')
    for k, v in tqdm(preds_paths_map.items())
)

100%|██████████| 3/3 [00:05<00:00,  2.00s/it]


In [291]:
def prepare_data_for_indexing(df, col_with_preds=None, max_preds=None, col_with_text='title_and_text'):
    to_concat = []
    if col_with_text is not None:
        to_concat.append(col_with_text)
    if col_with_preds is not None:
        preds_expr = pl.col(col_with_preds)
        if max_preds is not None:
            preds_expr = preds_expr.arr.slice(0, max_preds)
        to_concat.append(preds_expr.arr.join(' '))

    df = df.select(
        pl.col('_id').alias('id'),
        pl.concat_str(to_concat, separator= ' ').alias('contents'))
    return df

prepare_data_for_indexing(df_corpus_preds, 'preds_one_line_per_document_topk_sampling').head(1)

id,contents
str,str
"""ug7v899j""","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (3…"


In [294]:
index_build_fns = {
    'title_and_text_no_expansion': functools.partial(
        prepare_data_for_indexing,
        col_with_preds=None,
        max_preds=None,
        col_with_text='title_and_text'),
    'title_and_text_one_line_per_document_beam_search_1_pred': functools.partial(
        prepare_data_for_indexing,
        col_with_preds='preds_one_line_per_document_beam_search',
        max_preds=1,
        col_with_text='title_and_text'),
    'title_and_text_one_line_per_document_greedy_decode_1_pred': functools.partial(
        prepare_data_for_indexing,
        col_with_preds='preds_one_line_per_document_greedy_decode',
        max_preds=1,
        col_with_text='title_and_text'),
    'only_preds_one_line_per_document_beam_search_1_pred': functools.partial(
        prepare_data_for_indexing,
        col_with_preds='preds_one_line_per_document_beam_search',
        max_preds=1,
        col_with_text=None),
    'only_preds_one_line_per_document_greedy_decode_1_pred': functools.partial(
        prepare_data_for_indexing,
        col_with_preds='preds_one_line_per_document_greedy_decode',
        max_preds=1,
        col_with_text=None)
}